# 曲率计算代码

这个代码使用在以度规为已知条件，使用符号计算的方式，得到特定度规和曲率张量的关系。

In [38]:
import sympy
from sympy import sin, cos, log, exp

x_num = 4
x_list = sympy.symarray('x', x_num)

x_0 = x_list[0]
x_1 = x_list[1]
x_2 = x_list[2]
H = sympy.Function("H")
H = H(x_0, x_1, x_2)


# 度规张量
G_down = sympy.Array(
        [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, H, 1],
        [0, 0, 1, 0]])


G_up = sympy.Array(G_down.tomatrix().inv())

# 求度规张量的导数
G_diff = sympy.derive_by_array(G_down, x_list)

G_diff

[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, Derivative(H(x_0, x_1, x_2), x_0), 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, Derivative(H(x_0, x_1, x_2), x_1), 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, Derivative(H(x_0, x_1, x_2), x_2), 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]

In [39]:
# 计算克里斯多菲符号
Lambda = [[[0] * x_num for i in range(x_num)] for j in range(x_num)]

for i in range(x_num):
    for j in range(x_num):
        for k in range(x_num):
            tensor = sympy.tensorproduct(G_up[i, :], (G_diff[k, j, :] + G_diff[j, k, :] - G_diff[:, j, k]))
            Lambda[i][j][k] = sympy.tensorcontraction(tensor, (0, 1))

Lambda = sympy.simplify(sympy.Array(Lambda))
Lambda

[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, -Derivative(H(x_0, x_1, x_2), x_0), 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, -Derivative(H(x_0, x_1, x_2), x_1), 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, Derivative(H(x_0, x_1, x_2), x_0), 0], [0, 0, Derivative(H(x_0, x_1, x_2), x_1), 0], [Derivative(H(x_0, x_1, x_2), x_0), Derivative(H(x_0, x_1, x_2), x_1), Derivative(H(x_0, x_1, x_2), x_2), 0], [0, 0, 0, 0]]]

In [40]:
# 根据克里斯多菲符号计算曲率
Lambda_diff = sympy.derive_by_array(Lambda, x_list)

R = [[[[0] * x_num for i in range(x_num)] for j in range(x_num)] for k in range(x_num)]

for i in range(x_num):
    for j in range(x_num):
        for k in range(x_num):
            for m in range(x_num):
                R[i][j][k][m] += Lambda_diff[k][i][j][m] - Lambda_diff[m][i][j][k]
                for n in range(x_num):
                    R[i][j][k][m] += Lambda[i][n][k] * Lambda[n][j][m] - Lambda[i][n][m] * Lambda[n][k][j]

R = sympy.simplify(sympy.Array(R))
R

[[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, -Derivative(H(x_0, x_1, x_2), (x_0, 2)), 0], [0, 0, -Derivative(H(x_0, x_1, x_2), x_0, x_1), 0], [Derivative(H(x_0, x_1, x_2), (x_0, 2)), Derivative(H(x_0, x_1, x_2), x_0, x_1), 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]], [[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, -Derivative(H(x_0, x_1, x_2), x_0, x_1), 0], [0, 0, -Derivative(H(x_0, x_1, x_2), (x_1, 2)), 0], [Derivative(H(x_0, x_1, x_2), x_0, x_1), Derivative(H(x_0, x_1, x_2), (x_1, 2)), 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]], [[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [41]:
# 计算里奇张量
R1 = sympy.tensorcontraction(R, (0, 2))
R1

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, -Derivative(H(x_0, x_1, x_2), (x_0, 2)) - Derivative(H(x_0, x_1, x_2), (x_1, 2)), 0], [0, 0, 0, 0]]